# NOAA Storm Events 2023 - Data Exploration

**Phase 2 - Data Collection and Initial Analysis**

This notebook contains our initial exploration of the NOAA Storm Events 2023 dataset, which forms the foundation of our AmFam Risk Model project.

## Key Findings Summary

- **Dataset Size**: 75,593 storm events across 51 columns
- **Coverage**: Complete 2023 calendar year
- **Geographic**: County-level data (CZ_TYPE: C = County, Z = Zone)
- **Damage Format**: String format like '25.00K', '1.00M', '100.00M' requiring parsing
- **Event Types**: 51 different weather event types, dominated by Thunderstorm Wind and Hail

## Business Context

This dataset will feed into our:
1. **Risk Tier Classifier**: Predicting Low/Moderate/High/Extreme risk levels by county
2. **Damage Regressor**: Estimating expected annual property damage ($) by county
3. **Feature Engineering**: Aggregating event-level data to county-level statistics


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load the data
data_path = Path('../data/01_raw/noaa/StormEvents_details_2023.csv')
df = pd.read_csv(data_path)

print(f"Dataset loaded: {len(df):,} rows x {len(df.columns)} columns")
print(f"File size: {data_path.stat().st_size / 1024 / 1024:.1f} MB")

## Dataset Overview

In [ ]:
# Basic dataset info
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)

print(f"\nShape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"Date range: {df['YEAR'].min()} (all {df['YEAR'].nunique()} year(s))")

# Show all column names
print(f"\nColumn Names ({len(df.columns)} total):")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")

## Key Categorical Variables

In [ ]:
# Event Types Analysis
print("EVENT TYPES - Critical for Risk Classification")
print("=" * 50)

event_counts = df['EVENT_TYPE'].value_counts()
print(f"Total unique event types: {len(event_counts)}")
print(f"\nTop 15 Most Frequent Event Types:")
print(event_counts.head(15).to_string())

# Calculate percentage of top events
top_5_pct = (event_counts.head(5).sum() / len(df)) * 100
print(f"\nTop 5 event types represent {top_5_pct:.1f}% of all events")

In [ ]:
# County Zone Types
print("COUNTY/ZONE TYPES (CZ_TYPE)")
print("=" * 30)

cz_counts = df['CZ_TYPE'].value_counts()
print(cz_counts.to_string())
print(f"\nC = County, Z = Zone")
print(f"County events: {cz_counts['C']:,} ({cz_counts['C']/len(df)*100:.1f}%)")
print(f"Zone events: {cz_counts['Z']:,} ({cz_counts['Z']/len(df)*100:.1f}%)")

## Geographic Distribution

In [ ]:
# Geographic analysis
print("GEOGRAPHIC DISTRIBUTION")
print("=" * 25)

state_counts = df['STATE'].value_counts()
print(f"States covered: {len(state_counts)}")
print(f"\nTop 10 States by Event Count:")
print(state_counts.head(10).to_string())

print(f"\nUnique counties/zones: {df['CZ_NAME'].nunique():,}")

## Damage Analysis - Critical for AmFam Business Case

In [ ]:
# Property damage analysis
print("PROPERTY DAMAGE ANALYSIS")
print("=" * 30)

print(f"Total events: {len(df):,}")
print(f"Events with property damage data: {df['DAMAGE_PROPERTY'].count():,}")
print(f"Events with null property damage: {df['DAMAGE_PROPERTY'].isnull().sum():,}")
print(f"Coverage: {df['DAMAGE_PROPERTY'].count()/len(df)*100:.1f}%")

print(f"\nUnique damage values: {df['DAMAGE_PROPERTY'].nunique():,}")

# Show damage format examples
print(f"\nDAMAGE FORMAT EXAMPLES (Need Parsing):")
damage_examples = ['0.00K', '1.00K', '5.00K', '25.00K', '100.00K', '500.00K', '1.00M', '5.00M', '100.00M']
found_examples = [ex for ex in damage_examples if ex in df['DAMAGE_PROPERTY'].values]
for i, ex in enumerate(found_examples, 1):
    print(f"{i:2}. '{ex}'")

# Most common values
print(f"\nMost Frequent Damage Values:")
print(df['DAMAGE_PROPERTY'].value_counts().head(10).to_string())

## Data Quality Assessment

In [ ]:
# Missing data analysis
print("DATA QUALITY - NULL VALUE ANALYSIS")
print("=" * 40)

null_counts = df.isnull().sum()
null_pct = (null_counts / len(df)) * 100

# Only show columns with missing data
missing_data = pd.DataFrame({
    'Column': null_counts[null_counts > 0].index,
    'Missing_Count': null_counts[null_counts > 0].values,
    'Missing_Percentage': null_pct[null_counts > 0].values
}).sort_values('Missing_Count', ascending=False)

print(f"Columns with missing data: {len(missing_data)} out of {len(df.columns)}")
print(f"\nTop 15 columns by missing values:")
print(missing_data.head(15).to_string(index=False))

In [ ]:
# Key columns for risk modeling - completeness check
print("KEY COLUMNS FOR RISK MODELING")
print("=" * 35)

key_columns = ['STATE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'EVENT_TYPE', 
               'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'INJURIES_DIRECT', 
               'DEATHS_DIRECT', 'BEGIN_LAT', 'BEGIN_LON']

for col in key_columns:
    if col in df.columns:
        null_count = df[col].isnull().sum()
        null_pct = (null_count / len(df)) * 100
        print(f"{col:20}: {null_count:6,} nulls ({null_pct:5.1f}%)")

## Sample Data for Understanding Structure

In [ ]:
# Show first few rows with key columns for understanding
key_display_cols = ['STATE', 'CZ_NAME', 'EVENT_TYPE', 'DAMAGE_PROPERTY', 
                   'BEGIN_DATE_TIME', 'EVENT_NARRATIVE']

print("SAMPLE DATA - Key Columns")
print("=" * 30)
display_df = df[key_display_cols].head()
for idx, row in display_df.iterrows():
    print(f"\nRow {idx + 1}:")
    for col in key_display_cols:
        value = str(row[col])[:100] + "..." if len(str(row[col])) > 100 else str(row[col])
        print(f"  {col:15}: {value}")

## Next Steps for Silver Layer

Based on this exploration, our Bronze → Silver transformation pipeline needs to:

### Data Cleaning Requirements
1. **Parse damage values**: Convert '25.00K' → 25000, '1.00M' → 1000000
2. **Standardize event types**: Group similar events (e.g., various wind events)
3. **Handle missing coordinates**: Impute or flag events without lat/lon
4. **Date standardization**: Ensure consistent datetime formats
5. **County consolidation**: Focus on CZ_TYPE='C' for county-level analysis

### Feature Engineering for Gold Layer
1. **County-level aggregation**: Sum damages, count events by type per county
2. **Risk indicators**: Create severity scores based on damage/injury/death patterns
3. **Temporal features**: Seasonality, trends, event frequency patterns
4. **Geographic features**: Regional risk patterns, neighboring county effects

### Model Input Preparation
- Target variables: Risk tier (Low/Moderate/High/Extreme) + Expected damage ($)
- Features: County weather patterns, historical damage, event frequency by type
- Granularity: County-year level for training data